In [ ]:
import pandas as pd
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [ ]:
######### data import and processing

import pandas as pd

# Load the data
file_path = '/content/Data_Combined_2023-2024_daily.csv'
data = pd.read_csv(file_path)

# Drop the unnecessary index column
data.drop(columns=['Unnamed: 0'], inplace=True)

# Convert date column to datetime format and set as index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Drop the Gas Demand column
data.drop(columns=['Gas Demand'], inplace=True)

# Handle missing values
data['Gas Price'].fillna(method='ffill', inplace=True)
data['interconn_fra'].fillna(method='ffill', inplace=True)

# Forward fill first, then backward fill
data['CO2_Value'].fillna(method='ffill', inplace=True)
data['CO2_Value'].fillna(method='bfill', inplace=True)

data.rename(columns={'Electricity Demand': 'Electricity_Demand', 'Gas Price': 'Gas_Price', 'Electricity Price': 'Electricity_Price'}, inplace=True)


# Check for remaining missing values
missing_values_after = data.isnull().sum()
print("Missing values after cleaning:")
print(missing_values_after)



In [ ]:
############ BASE MODEL

# Define the target and independent variables
target = 'Electricity_Price'
independent_vars = data.columns.drop(target).tolist()
X = data[independent_vars]
y = data[target]

# Fit the ARIMAX model
from statsmodels.tsa.statespace.sarimax import SARIMAX

order = (1, 1, 1)  # ARIMA order (p, d, q), these should be tuned based on your data
seasonal_order = (0, 0, 0, 0)  # No seasonality for ARIMAX

model = SARIMAX(y, exog=X, order=order, seasonal_order=seasonal_order)
results = model.fit(disp=False)

# Summary of the model
print(results.summary())

# Model diagnostics and forecasting
import matplotlib.pyplot as plt

# Diagnostics
results.plot_diagnostics(figsize=(15, 12))
plt.show()

# Forecast
forecast_steps = 30  # Number of days to forecast
forecast = results.get_forecast(steps=forecast_steps, exog=X[-forecast_steps:])

# Corrected forecast index generation
forecast_index = pd.date_range(start=X.index[-1] + pd.Timedelta(days=1), periods=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

plt.figure(figsize=(10, 5))
plt.plot(y, label='Observed')
plt.plot(forecast_index, forecast_mean, label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='k', alpha=0.1)
plt.legend()
plt.show()

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd

# Prepare the data
data_prophet = data.reset_index().rename(columns={'Date': 'ds', 'Electricity_Price': 'y'})

# Create lagged features, moving averages, and interaction terms
data_prophet['lag_1'] = data_prophet['y'].shift(1)
data_prophet['lag_7'] = data_prophet['y'].shift(7)
data_prophet['ma_7'] = data_prophet['y'].rolling(window=7).mean()
data_prophet['ma_30'] = data_prophet['y'].rolling(window=30).mean()

# Additional interaction terms
data_prophet['interaction_1'] = data_prophet['Gas_Price'] * data_prophet['CO2_Value']
data_prophet['interaction_2'] = data_prophet['Temperature'] * data_prophet['Electricity_Demand']
data_prophet['interaction_3'] = data_prophet['Gas Price'] * data_prophet['Electricity_Demand']
data_prophet['interaction_4'] = data_prophet['Temperature'] * data_prophet['Gas_Price']

# Drop NaN values created by shifting and rolling calculations
data_prophet.dropna(inplace=True)

# Split into training and testing sets
train_size = int(len(data_prophet) * 0.8)
train_data, test_data = data_prophet[:train_size], data_prophet[train_size:]

# Hyperparameter grid (extended)
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'yearly_seasonality': [True, False],
    'weekly_seasonality': [True, False],
    'daily_seasonality': [False],  # Typically, daily seasonality is not necessary for daily data
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5, 1.0],
}

# List of regressors (defined globally)
regressors = ['Temperature', 'Electricity_Demand', 'Gas_Price', 'interconn_fra', 'CO2_Value'] + \
             ['lag_1', 'lag_7', 'ma_7', 'ma_30', 'interaction_1', 'interaction_2', 'interaction_3', 'interaction_4']

# Function to train and evaluate a model with given parameters
def evaluate_prophet(params):
    model = Prophet(
        seasonality_mode=params['seasonality_mode'],
        yearly_seasonality=params['yearly_seasonality'],
        weekly_seasonality=params['weekly_seasonality'],
        daily_seasonality=params['daily_seasonality'],
        seasonality_prior_scale=params['seasonality_prior_scale'],
        changepoint_prior_scale=params['changepoint_prior_scale']
    )
    for regressor in regressors:
        model.add_regressor(regressor)

    # Fit the model
    model.fit(train_data)

    # Perform cross-validation
    df_cv = cross_validation(model, initial='180 days', period='90 days', horizon='180 days')
    df_p = performance_metrics(df_cv)

    return df_p['rmse'].mean()

# Perform grid search
best_params = None
best_score = float('inf')
for seasonality_mode in param_grid['seasonality_mode']:
    for yearly_seasonality in param_grid['yearly_seasonality']:
        for weekly_seasonality in param_grid['weekly_seasonality']:
            for daily_seasonality in param_grid['daily_seasonality']:
                for seasonality_prior_scale in param_grid['seasonality_prior_scale']:
                    for changepoint_prior_scale in param_grid['changepoint_prior_scale']:
                        params = {
                            'seasonality_mode': seasonality_mode,
                            'yearly_seasonality': yearly_seasonality,
                            'weekly_seasonality': weekly_seasonality,
                            'daily_seasonality': daily_seasonality,
                            'seasonality_prior_scale': seasonality_prior_scale,
                            'changepoint_prior_scale': changepoint_prior_scale
                        }
                        score = evaluate_prophet(params)
                        if score < best_score:
                            best_score = score
                            best_params = params
                        print(f'Params: {params}, RMSE: {score}')

print(f'Best Params: {best_params}, Best RMSE: {best_score}')

# Train the final model with the best parameters
model2 = Prophet(
    seasonality_mode=best_params['seasonality_mode'],
    yearly_seasonality=best_params['yearly_seasonality'],
    weekly_seasonality=best_params['weekly_seasonality'],
    daily_seasonality=best_params['daily_seasonality'],
    seasonality_prior_scale=best_params['seasonality_prior_scale'],
    changepoint_prior_scale=best_params['changepoint_prior_scale']
)
for regressor in regressors:
    model2.add_regressor(regressor)

# Fit the model
model2.fit(train_data)

# Make predictions
future = model2.make_future_dataframe(periods=len(test_data))
for regressor in regressors:
    future[regressor] = data_prophet[regressor]

# Fill NaN values in future with the last available values
for col in regressors:
    future[col].fillna(method='ffill', inplace=True)
    future[col].fillna(method='bfill', inplace=True)

forecast = model2.predict(future)

# Evaluate the performance
y_pred = forecast['yhat'][train_size:]
y_true = test_data['y']

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_true, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}')

# Plot the forecast
fig = model2.plot(forecast)
plt.show()

# Plot the forecast components
fig2 = model2.plot_components(forecast)
plt.show()

# Perform cross-validation on the final model
df_cv = cross_validation(model2, initial='180 days', period='90 days', horizon='180 days')
df_p = performance_metrics(df_cv)

print(df_p[['horizon', 'mae', 'mse', 'rmse', 'mape']])

# Plot cross-validation results
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_cv['ds'], df_cv['y'], 'k.', label='Actual')
ax.plot(df_cv['ds'], df_cv['yhat'], 'b-', label='Predicted')
ax.fill_between(df_cv['ds'].dt.to_pydatetime(), df_cv['yhat_lower'], df_cv['yhat_upper'], color='blue', alpha=0.2)
plt.legend()
plt.show()



##model is still better than model 1 and 2 and the best parameters are
#Best Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': True, 'daily_seasonality': False, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.01}


#lets try arimax again


In [ ]:
########## THIS USED TO FIND THE BEST PARAMETERS
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Prepare the data
data_prophet = data.reset_index().rename(columns={'Date': 'ds', 'Electricity_Price': 'y'})

# Create lagged features, moving averages, and interaction terms
data_prophet['lag_1'] = data_prophet['y'].shift(1)
data_prophet['lag_7'] = data_prophet['y'].shift(7)
data_prophet['ma_7'] = data_prophet['y'].rolling(window=7).mean()
data_prophet['ma_30'] = data_prophet['y'].rolling(window=30).mean()

# Additional interaction terms
data_prophet['interaction_1'] = data_prophet['Gas_Price'] * data_prophet['CO2_Value']
data_prophet['interaction_2'] = data_prophet['Temperature'] * data_prophet['Electricity_Demand']
data_prophet['interaction_3'] = data_prophet['Gas_Price'] * data_prophet['Electricity_Demand']
data_prophet['interaction_4'] = data_prophet['Temperature'] * data_prophet['Gas_Price']

# Drop NaN values created by shifting and rolling calculations
data_prophet.dropna(inplace=True)

# Define the target and independent variables
target = 'y'
independent_vars = ['Temperature', 'Electricity_Demand', 'Gas_Price', 'interconn_fra', 'CO2_Value',
                    'lag_1', 'lag_7', 'ma_7', 'ma_30', 'interaction_1', 'interaction_2', 'interaction_3', 'interaction_4']
X = data_prophet[independent_vars]
y = data_prophet[target]

# Split into training and testing sets
train_size = int(len(data_prophet) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define a larger grid search for ARIMA order and seasonal order
p = d = q = range(0, 3)
seasonal_p = seasonal_d = seasonal_q = range(0, 2)
seasonal_period = [0, 7, 30]

# Generate all different combinations of (p, d, q) and seasonal (P, D, Q, s)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(seasonal_p, seasonal_d, seasonal_q, seasonal_period))

# Function to evaluate ARIMAX model
def evaluate_arimax(order, seasonal_order, exog, endog):
    try:
        model = SARIMAX(endog, exog=exog, order=order, seasonal_order=seasonal_order)
        results = model.fit(disp=False)
        y_pred = results.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return rmse
    except:
        return float('inf')

# Perform grid search
best_score = float('inf')
best_params = None
for param in pdq:
    for param_seasonal in seasonal_pdq:
        score = evaluate_arimax(param, param_seasonal, X_train, y_train)
        if score < best_score:
            best_score = score
            best_params = (param, param_seasonal)
        print(f'ARIMA{param}x{param_seasonal}, RMSE: {score}')

print(f'Best Params: ARIMA{best_params[0]}x{best_params[1]}, Best RMSE: {best_score}')

# Train the final model with the best parameters
model3 = SARIMAX(y, exog=X, order=best_params[0], seasonal_order=best_params[1])
results3 = model3.fit(disp=False)

# Summary of the model
print(results3.summary())

# Model diagnostics and forecasting
results3.plot_diagnostics(figsize=(15, 12))
plt.show()

# Forecast
forecast_steps = 30  # Number of days to forecast
forecast = results3.get_forecast(steps=forecast_steps, exog=X[-forecast_steps:])

# Corrected forecast index generation
forecast_index = pd.date_range(start=X.index[-1] + pd.Timedelta(days=1), periods=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

plt.figure(figsize=(10, 5))
plt.plot(y, label='Observed')
plt.plot(forecast_index, forecast_mean, label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='k', alpha=0.1)
plt.legend()
plt.show()

# Evaluate the performance on the test set
y_pred = results3.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}')


In [ ]:
#########FINAL MODEL SARIMAX

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt


# Define the target and independent variables
target = 'Electricity_Price'
independent_vars = data.columns.drop(target).tolist()
X = data[independent_vars]
y = data[target]


# Prepare the data (assuming data is already loaded in a DataFrame called 'data')
data_prophet = data.reset_index().rename(columns={'Date': 'ds', 'Electricity_Price': 'y'})



# Create lagged features, moving averages, and interaction terms
data_prophet['lag_1'] = data_prophet['y'].shift(1)
data_prophet['lag_7'] = data_prophet['y'].shift(7)
data_prophet['ma_7'] = data_prophet['y'].rolling(window=7).mean()
data_prophet['ma_30'] = data_prophet['y'].rolling(window=30).mean()

data_prophet['interaction_1'] = data_prophet['Gas_Price'] * data_prophet['CO2_Value']
data_prophet['interaction_2'] = data_prophet['Temperature'] * data_prophet['Electricity_Demand']
data_prophet['interaction_3'] = data_prophet['Gas_Price'] * data_prophet['Electricity_Demand']
data_prophet['interaction_4'] = data_prophet['Temperature'] * data_prophet['Gas_Price']

data_prophet.dropna(inplace=True)

# Define the target and independent variables
target = 'y'
independent_vars = ['Temperature', 'Electricity_Demand', 'Gas_Price', 'interconn_fra', 'CO2_Value',
                    'lag_1', 'lag_7', 'ma_7', 'ma_30', 'interaction_1', 'interaction_2', 'interaction_3', 'interaction_4']
X = data_prophet[independent_vars]
y = data_prophet[target]

# Split into training and testing sets
train_size = int(len(data_prophet) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Best parameters from previous grid search
best_order = (0, 0, 0)
best_seasonal_order = (1, 0, 0, 7)

# Train the final model with the best parameters
model3 = SARIMAX(y_train, exog=X_train, order=best_order, seasonal_order=best_seasonal_order)
results3 = model3.fit(disp=False)

# Summary of the model
print(results3.summary())

# Model diagnostics
results3.plot_diagnostics(figsize=(15, 12))
plt.show()

# Forecast future values
forecast_steps = len(y_test)  # Number of steps to forecast
forecast = results3.get_forecast(steps=forecast_steps, exog=X_test)

# Generate forecast index
forecast_index = y_test.index
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Plot forecast
plt.figure(figsize=(10, 5))
plt.plot(y_train, label='Train')
plt.plot(y_test, label='Test')
plt.plot(forecast_index, forecast_mean, label='Forecast')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='k', alpha=0.1)
plt.legend()
plt.show()

# Evaluate the performance on the test set
y_pred = results3.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}')


In [ ]:
mape = 0.17347834046876742
accuracy = 100 - (mape * 100)
print(f'Accuracy: {accuracy:.2f}%')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import joblib

# Assuming 'model3' is your ARIMAX model fitted with the best parameters
model3_file_path = '/content/drive/My Drive/Electricity_Price_model.pkl'
joblib.dump(results3, model3_file_path)
print(f'Model saved to {model3_file_path}')


# Models for independent variables

In [ ]:
# rename columns
data.rename(columns={'Electricity Demand': 'Electricity_Demand', 'Gas Price': 'Gas_Price' }, inplace=True)

In [ ]:
def create_lagged_dataframe(data, var_name):
    # Initialize the dataframe with the original column
    lagged_df = data[[var_name]].copy()

    # Create lagged columns from lag1 to lag15
    for i in range(1, 16):
        lagged_df[f'{var_name}_lag{i}'] = data[var_name].shift(i)

    # Remove the first 15 rows that contain NaN values
    lagged_df = lagged_df.dropna()

    return lagged_df

In [ ]:
CO2_Value_lags = create_lagged_dataframe(data, 'Electricity_Demand')

In [ ]:
# write csv
CO2_Value_lags.to_csv('Electricity_Demand_lags.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

def train_and_plot_regression(data, var_name):
    # Create the lagged dataframe
    lagged_df = create_lagged_dataframe(data, var_name)

    # Prepare the features and target variable
    X = lagged_df.drop(columns=[var_name])
    y = lagged_df[var_name]

    # Determine the split index for 80/20 split
    split_index = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
    test_dates = y_test.index  # Keeping the date index for plotting

    # Initialize and train the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict the target variable on the test set
    y_pred = model.predict(X_test)

    # Plotting the results
    plt.figure(figsize=(10, 6))
    plt.plot(test_dates, y_test, label='Actual', color='blue', marker='o')
    plt.plot(test_dates, y_pred, label='Predicted', color='red', linestyle='--', marker='x')
    plt.title(f'Actual vs Predicted {var_name} Values Over Time')
    plt.xlabel('Date')
    plt.ylabel(f'{var_name}')
    plt.legend()
    plt.xticks(rotation=45)  # Rotate date labels for better readability
    plt.tight_layout()  # Adjust layout to make room for label rotation
    plt.show()

    # Optional: Return model, scores, or coefficients if needed
    return model, X_train, X_test, y_train, y_test, y_pred

In [ ]:
model, X_train, X_test, y_train, y_test, y_pred = train_and_plot_regression(data, 'CO2_Value')

In [ ]:
for feature, coef in zip(X_train.columns, model.coef_):
    print(f"{feature}: {coef}")

In [ ]:
import joblib

model_file_path = '/content/drive/My Drive/CO2_Value_model.pkl'
joblib.dump(model, model_file_path)
print(f'Model saved to {model_file_path}')

In [ ]:
from datetime import timedelta
import pandas as pd
from joblib import load

def predict_future_values(var_names, end_date, data):
    for var_name in var_names:
        # Load the model for the specified variable
        model = load(f"{var_name}_model.pkl")
        var_lags = pd.read_csv(f'{var_name}_lags.csv', index_col='Date', parse_dates=True)

        # Convert end_date to pandas Timestamp
        end_date = pd.to_datetime(end_date)

        # Make sure the index is in datetime format and the DataFrame is sorted by index
        var_lags.index = pd.to_datetime(var_lags.index)
        var_lags = var_lags.sort_index()

        # Start predicting from the day after the latest date in var_lags
        current_date = var_lags.index.max() + timedelta(days=1)

        while current_date <= end_date:
            # Extract the last 15 entries to calculate lags
            last_entries = var_lags.tail(15)
            new_row = {f'{var_name}_lag{i}': last_entries[var_name].shift(i-1).iloc[-1] for i in range(1, 16)}

            # Prepare features and predict
            features = [new_row[f'{var_name}_lag{i}'] for i in range(1, 16)]
            predicted_value = model.predict([features])[0]

            # Append new row to DataFrame with current_date as the index
            new_row[var_name] = predicted_value
            var_lags.loc[current_date] = new_row

            # Add the prediction to the data DataFrame
            if var_name not in data.columns:
                data[var_name] = pd.NA
            data.loc[current_date, var_name] = predicted_value

            # Increment the date by one day
            current_date += timedelta(days=1)

    return data

In [ ]:
end_date = '2024-06-30'
var_names = ['Temperature', 'Electricity_Demand', 'Gas_Price', 'CO2_Value', 'interconn_fra']
data_predicted = predict_future_values(var_names, end_date, data)